In [9]:
from langchain.llms.openai import OpenAI
from langchain.chat_models.openai import ChatOpenAI

llm = OpenAI(model_name="gpt-3.5-turbo-instruct")
chat = ChatOpenAI()

a = llm.predict("How many planets are there?")
b = chat.predict("How many planets are there?")

a, b

('\n\nThere are eight planets in our solar system: Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, and Neptune.',
 'There are 8 planets in our solar system: Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, and Neptune.')

In [10]:
from langchain.schema import HumanMessage, AIMessage, SystemMessage

# Pass settings to AI via SystemMessage. Ask questions via HumanMessage
messages = [
    SystemMessage(content="You are a geography expert. And you only reply in Korean. Also use 반말"),
    AIMessage(content="안녕!"),
    HumanMessage(content="What is the distance between Korea and New York.")
]

chat.predict_messages(messages)

AIMessage(content='한국과 뉴욕 사이의 거리는 대략 11,000km 정도 됩니다.')

In [1]:
from langchain.prompts import PromptTemplate, ChatPromptTemplate

template = ChatPromptTemplate.from_messages([
    ("system", "You are a geography expert. And you only reply in {language}."),
    ("ai", "My name is {name}."),
    ("human", "What is the distance between {country_a} and {country_b}. Also, what is your name?")
])

prompt = template.format_messages(
    language="English",
    name="Socrates",
    country_a="Korea", 
    country_b="New York, United States")

chat.predict_messages(prompt)


NameError: name 'chat' is not defined

In [4]:
from langchain.schema import BaseOutputParser

class CommaOutputParser(BaseOutputParser):
    def parse(self, text):
        items = text.strip().split(",")
        return list(map(str.strip,items))
    

p = CommaOutputParser()

p.parse("Hello,how,are,you")

['Hello', 'how', 'are', 'you']

In [13]:
template = ChatPromptTemplate.from_messages([
    ("system","You are a list generating machine. Everything you are asked will be answered with a comma-separated list of maximum {max_items} in lower case. Do NOT reply with anything else."),
    ("human", "{question}"),
])

prompt = template.format_messages(
    max_items=10,
    question="What are the planets?"
)

result = chat.predict_messages(prompt)

p = CommaOutputParser()
p.parse(result.content)

['mercury',
 'venus',
 'earth',
 'mars',
 'jupiter',
 'saturn',
 'uranus',
 'neptune',
 'pluto']

In [14]:
chat = ChatOpenAI()

template = ChatPromptTemplate.from_messages([
    ("system","You are a list generating machine. Everything you are asked will be answered with a comma-separated list of maximum {max_items} in lower case. Do NOT reply with anything else."),
    ("human", "{question}"),
])

prompt = template.format_messages(
    max_items=10,
    question="What are the planets?"
)

# | creates chain.
chain = template | chat | CommaOutputParser()
chain.invoke({
    "max_items":5,
    "question": "What are the pokemons?"
})

['pikachu', 'charmander', 'bulbasaur', 'squirtle', 'jigglypuff']